**SQL Server Integration

In [ ]:
# Imports de librerias
import pyodbc
import openai
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import json

In [ ]:
# API KEY de OPNEAI
os.environ["OPENAI_API_KEY"] = "xxxxxx"

In [ ]:
# Conexión con base de datos
connectionSQLSever = pyodbc.connect("DRIVER={SQL Server};SERVER=xxxxxx;DATABASE=xxxxxx;Trusted_Connection=yes;")

In [ ]:
# Variable de conexión y variable para ejecutar scripts SQL
connection = connectionSQLSever
cursor = connection.cursor()

In [ ]:
# Variable que aloja la ruta del documento pdf
rutaPdfs = ["xxxxxx"]

In [ ]:
def lecturaPdfs(rutaPdfs):
    texts = []
    texto = ""
    all_chunks = []
    
    for pdf in rutaPdfs:
        pdf_reader = PdfReader(pdf)
        pdf_text = ''
        for page in pdf_reader.pages:
            pdf_text += page.extract_text()
        texts.append(pdf_text)
        texto += pdf_text   
         
        text_splitter = CharacterTextSplitter(
            separator="\n",
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
)
    chunks = text_splitter.split_text(pdf_text)
    all_chunks.extend(chunks)
        
    return texto, all_chunks

In [ ]:
texto, all_chunks = lecturaPdfs(rutaPdfs)

In [ ]:
openai.api_key= "xxxxxxx"

response = openai.embeddings.create(model="text-embedding-ada-002", input=all_chunks)


In [ ]:
response.data[0].embedding

In [ ]:
embedding_data = response.data[0].embedding
embedding_json = json.dumps(embedding_data)
# Encode json a UTF-8
embedding_bytes = embedding_json.encode('utf-8')

In [ ]:
queryInsertAllChunks = """
    INSERT INTO Vector (vector) VALUES (?);
"""
queryInsertTexto = """
    INSERT INTO Vector (vector) VALUES (?);
"""
cursor.execute(queryInsertTexto, ((texto),))

for chunk in all_chunks:
    cursor.execute(queryInsertAllChunks, (chunk,))

In [ ]:
connection.commit()

In [ ]:

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0.5, streaming=True
)


qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        verbose=True,
        return_source_documents=True,

)


In [ ]:
embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(texts=chunk, embedding=embeddings)
retriever = docsearch.as_retriever()

In [ ]:
chunk

In [ ]:
result = qa({"query":"xxxxxx"})